# Advanced Optimization Topics and Case Studies

This lesson synthesizes optimization concepts through advanced techniques and comprehensive case studies. We'll tackle real-world complexity including uncertainty, multi-objective optimization, and large-scale decomposition methods. The focus is on developing strategies for complex power system challenges that go beyond textbook formulations.

## Learning Objectives

By the end of this lesson, you will be able to:

1. Apply stochastic and robust optimization to handle uncertainty
2. Formulate and solve multi-objective optimization problems
3. Use decomposition techniques for large-scale problems
4. Integrate energy storage into optimization models
5. Solve transmission planning problems
6. Design optimization frameworks for future grid challenges

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pulp import *
import time
from scipy import stats
from concurrent.futures import ProcessPoolExecutor
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

## 1. Handling Uncertainty

Power systems face increasing uncertainty from renewable generation, demand variability, and equipment failures. We'll explore different approaches to optimization under uncertainty.

### Scenario-Based Stochastic Programming

Stochastic programming optimizes expected performance across multiple scenarios. The key is balancing computational tractability with adequate uncertainty representation.

In [ ]:
def generate_wind_scenarios(n_hours, n_scenarios, wind_capacity):
    """Generate wind power scenarios using statistical models."""
    scenarios = {}
    probabilities = []
    
    # Generate scenarios using truncated normal distribution
    for i in range(n_scenarios):
        # Base capacity factor varies by scenario
        base_cf = stats.truncnorm.rvs(0, 1, loc=0.35, scale=0.15)
        
        # Add hourly correlation
        hourly_noise = np.zeros(n_hours)
        hourly_noise[0] = np.random.normal(0, 0.05)
        
        for t in range(1, n_hours):
            # AR(1) process for temporal correlation
            hourly_noise[t] = 0.8 * hourly_noise[t-1] + np.random.normal(0, 0.05)
        
        capacity_factors = base_cf + hourly_noise
        capacity_factors = np.clip(capacity_factors, 0, 1)
        
        scenarios[f'S{i+1}'] = wind_capacity * capacity_factors
        probabilities.append(1.0 / n_scenarios)  # Equal probability
    
    return scenarios, probabilities

# Generate scenarios
wind_scenarios, wind_probs = generate_wind_scenarios(24, 10, 200)

# Visualize scenarios
plt.figure(figsize=(12, 6))
for name, profile in wind_scenarios.items():
    plt.plot(profile, alpha=0.5, label=name if 'S1' in name or 'S5' in name or 'S10' in name else '')

# Plot expected value
expected_wind = sum(prob * profile for profile, prob in zip(wind_scenarios.values(), wind_probs))
plt.plot(expected_wind, 'k-', linewidth=3, label='Expected')

plt.xlabel('Hour')
plt.ylabel('Wind Power (MW)')
plt.title('Wind Power Scenarios')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Scenario statistics:")
print(f"Mean capacity factor: {expected_wind.mean() / 200:.2%}")
print(f"Std deviation: {np.std([s.mean() for s in wind_scenarios.values()]):.1f} MW")

### Robust Optimization

Robust optimization ensures feasibility for all scenarios within an uncertainty set, providing guaranteed performance at the cost of conservatism.

In [ ]:
def robust_economic_dispatch(generators, demand, wind_forecast, uncertainty_budget):
    """
    Robust ED considering worst-case wind deviation.
    uncertainty_budget controls conservatism (0 = deterministic, 1 = worst case).
    """
    prob = LpProblem("Robust_ED", LpMinimize)
    
    # Variables
    p = {g: LpVariable(f"p_{g}", gen['p_min'], gen['p_max']) 
         for g, gen in generators.items()}
    
    # Wind deviation variables
    wind_min = wind_forecast * (1 - uncertainty_budget * 0.3)  # 30% max deviation
    wind_max = wind_forecast * (1 + uncertainty_budget * 0.1)  # 10% max upside
    
    # Auxiliary variable for robust constraint
    reserve = LpVariable("reserve", 0)
    
    # Objective
    prob += lpSum(generators[g]['cost'] * p[g] for g in generators)
    
    # Robust constraint: must meet demand for worst-case wind
    prob += lpSum(p[g] for g in generators) + wind_min >= demand
    
    # Reserve for wind uncertainty
    prob += reserve == wind_forecast - wind_min
    prob += lpSum(generators[g]['p_max'] - p[g] for g in generators) >= reserve
    
    prob.solve()
    
    return prob, p, reserve

# Example system
generators = {
    'G1': {'p_min': 50, 'p_max': 200, 'cost': 25},
    'G2': {'p_min': 40, 'p_max': 150, 'cost': 30},
    'G3': {'p_min': 30, 'p_max': 100, 'cost': 35}
}

demand = 300
wind_forecast = 80

# Compare different uncertainty budgets
budgets = [0, 0.25, 0.5, 0.75, 1.0]
results = []

for budget in budgets:
    prob, p, reserve = robust_economic_dispatch(generators, demand, wind_forecast, budget)
    if prob.status == 1:
        results.append({
            'budget': budget,
            'cost': value(prob.objective),
            'reserve': value(reserve),
            'dispatch': {g: value(p[g]) for g in generators}
        })

# Visualize robust optimization trade-off
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Cost vs conservatism
costs = [r['cost'] for r in results]
reserves = [r['reserve'] for r in results]

ax1.plot(budgets, costs, 'bo-', markersize=8)
ax1.set_xlabel('Uncertainty Budget')
ax1.set_ylabel('Total Cost ($)')
ax1.set_title('Cost of Robustness')
ax1.grid(True, alpha=0.3)

# Reserve requirements
ax2.bar(budgets, reserves, color='orange', alpha=0.7)
ax2.set_xlabel('Uncertainty Budget')
ax2.set_ylabel('Reserve Requirement (MW)')
ax2.set_title('Reserve vs Uncertainty Budget')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("Robust Optimization Results:")
print("Budget\tCost ($)\tReserve (MW)\tExtra Cost (%)")
base_cost = results[0]['cost']
for r in results:
    extra = (r['cost'] - base_cost) / base_cost * 100
    print(f"{r['budget']:.2f}\t{r['cost']:.0f}\t\t{r['reserve']:.1f}\t\t{extra:.1f}%")

### Exercise 1: Chance-Constrained Optimization

Implement chance-constrained optimization where constraints must be satisfied with a specified probability. This balances robustness with economic efficiency.

In [ ]:
# Exercise 1: Implement chance-constrained economic dispatch
# Requirements:
# - Demand must be met with 95% probability
# - Use scenario approximation for chance constraints
# - Compare with deterministic and robust solutions
# - Wind scenarios are provided

# Your implementation here

In [ ]:
# Solution
def chance_constrained_ed(generators, demand, wind_scenarios, wind_probs, confidence_level=0.95):
    """Chance-constrained ED using scenario approximation."""
    prob = LpProblem("Chance_Constrained_ED", LpMinimize)
    
    # First-stage variables (here-and-now decisions)
    p = {g: LpVariable(f"p_{g}", gen['p_min'], gen['p_max']) 
         for g, gen in generators.items()}
    
    # Binary variables for scenario violation
    z = {s: LpVariable(f"z_{s}", cat='Binary') for s in wind_scenarios}
    
    # Big-M for constraint relaxation
    M = 1000
    
    # Objective: minimize generation cost
    prob += lpSum(generators[g]['cost'] * p[g] for g in generators)
    
    # Chance constraint approximation
    for s_name, wind_s in wind_scenarios.items():
        # Demand must be met unless scenario is violated
        total_gen = lpSum(p[g] for g in generators)
        prob += total_gen + wind_s[12] >= demand - M * z[s_name]  # Hour 12 as example
    
    # Probability of meeting demand >= confidence level
    prob += lpSum(wind_probs[i] * (1 - z[s]) 
                  for i, s in enumerate(wind_scenarios)) >= confidence_level
    
    # Reserve constraint for expected wind
    expected_wind = sum(wind_probs[i] * wind_s[12] 
                       for i, wind_s in enumerate(wind_scenarios.values()))
    prob += lpSum(generators[g]['p_max'] - p[g] for g in generators) >= 0.1 * expected_wind
    
    prob.solve()
    
    return prob, p, z

# Generate more scenarios for chance constraint
wind_cc_scenarios, wind_cc_probs = generate_wind_scenarios(24, 20, 100)

# Solve with different confidence levels
confidence_levels = [0.8, 0.9, 0.95, 0.99]
cc_results = []

for conf in confidence_levels:
    prob_cc, p_cc, z_cc = chance_constrained_ed(generators, demand, 
                                                wind_cc_scenarios, wind_cc_probs, conf)
    
    if prob_cc.status == 1:
        # Calculate actual violation probability
        violations = sum(1 for s in wind_cc_scenarios if value(z_cc[s]) > 0.5)
        violation_prob = violations / len(wind_cc_scenarios)
        
        cc_results.append({
            'confidence': conf,
            'cost': value(prob_cc.objective),
            'violation_prob': violation_prob,
            'dispatch': {g: value(p_cc[g]) for g in generators}
        })

# Compare approaches
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Cost comparison
methods = ['Deterministic', 'Chance 90%', 'Chance 95%', 'Chance 99%', 'Robust']
costs_comp = [
    results[0]['cost'],  # Deterministic (uncertainty budget = 0)
    cc_results[1]['cost'],  # 90% confidence
    cc_results[2]['cost'],  # 95% confidence
    cc_results[3]['cost'],  # 99% confidence
    results[-1]['cost']  # Fully robust
]

bars = ax1.bar(methods, costs_comp, color=['blue', 'green', 'green', 'green', 'red'], alpha=0.7)
ax1.set_ylabel('Total Cost ($)')
ax1.set_title('Cost Comparison: Different Approaches')
ax1.grid(True, alpha=0.3, axis='y')

# Add cost increase labels
for i, (bar, cost) in enumerate(zip(bars, costs_comp)):
    height = bar.get_height()
    increase = (cost - costs_comp[0]) / costs_comp[0] * 100
    ax1.text(bar.get_x() + bar.get_width()/2., height + 5,
             f'+{increase:.1f}%', ha='center', va='bottom')

# Reliability vs cost trade-off
conf_levels = [r['confidence'] for r in cc_results]
cc_costs = [r['cost'] for r in cc_results]
actual_reliability = [1 - r['violation_prob'] for r in cc_results]

ax2.plot(conf_levels, cc_costs, 'bo-', label='Cost', markersize=8)
ax2.set_xlabel('Target Confidence Level')
ax2.set_ylabel('Cost ($)', color='b')
ax2.tick_params(axis='y', labelcolor='b')

ax2_twin = ax2.twinx()
ax2_twin.plot(conf_levels, actual_reliability, 'rs--', label='Actual Reliability', markersize=8)
ax2_twin.set_ylabel('Actual Reliability', color='r')
ax2_twin.tick_params(axis='y', labelcolor='r')
ax2_twin.set_ylim(0.7, 1.05)

ax2.set_title('Chance Constraints: Cost vs Reliability')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Chance-Constrained Optimization Results:")
print("Target\tActual\tCost ($)\tG1 (MW)\tG2 (MW)\tG3 (MW)")
for r in cc_results:
    print(f"{r['confidence']:.0%}\t{1-r['violation_prob']:.0%}\t{r['cost']:.0f}\t", end='')
    print(f"{r['dispatch']['G1']:.0f}\t{r['dispatch']['G2']:.0f}\t{r['dispatch']['G3']:.0f}")

## 2. Multi-Objective Optimization

Power system operation involves balancing multiple competing objectives. We'll explore techniques for handling trade-offs between cost, emissions, and reliability.

### Exercise 2: Pareto Frontier Analysis

Implement multi-objective optimization to explore the trade-off between operating cost and emissions. Generate and analyze the Pareto frontier.

In [ ]:
# Exercise 2: Multi-objective optimization with Pareto frontier
# Requirements:
# - Minimize both cost and emissions
# - Use epsilon-constraint method
# - Generate at least 10 Pareto-optimal points
# - Visualize trade-off curve and dispatch patterns

# Your implementation here

In [ ]:
# Solution
def multi_objective_ed(generators, demand, epsilon_emissions=None, weight_cost=None):
    """Multi-objective ED with cost and emissions."""
    prob = LpProblem("Multi_Objective_ED", LpMinimize)
    
    # Variables
    p = {g: LpVariable(f"p_{g}", gen['p_min'], gen['p_max']) 
         for g, gen in generators.items()}
    
    # Calculate objectives
    total_cost = lpSum(generators[g]['cost'] * p[g] for g in generators)
    total_emissions = lpSum(generators[g]['emissions'] * p[g] for g in generators)
    
    if epsilon_emissions is not None:
        # Epsilon-constraint method
        prob += total_cost
        prob += total_emissions <= epsilon_emissions
    elif weight_cost is not None:
        # Weighted sum method
        weight_emissions = 1 - weight_cost
        # Normalize objectives
        cost_norm = 15000  # Approximate scale
        emissions_norm = 400  # Approximate scale
        prob += weight_cost * (total_cost / cost_norm) + \
                weight_emissions * (total_emissions / emissions_norm)
    else:
        # Default: minimize cost only
        prob += total_cost
    
    # Constraints
    prob += lpSum(p[g] for g in generators) == demand
    
    prob.solve()
    
    if prob.status == 1:
        return {
            'status': 'optimal',
            'cost': value(total_cost),
            'emissions': value(total_emissions),
            'dispatch': {g: value(p[g]) for g in generators}
        }
    else:
        return {'status': 'infeasible'}

# Extended generator data with emissions
generators_mo = {
    'Coal1': {'p_min': 100, 'p_max': 400, 'cost': 22, 'emissions': 1.0},
    'Coal2': {'p_min': 80, 'p_max': 300, 'cost': 25, 'emissions': 0.95},
    'Gas1': {'p_min': 50, 'p_max': 250, 'cost': 30, 'emissions': 0.6},
    'Gas2': {'p_min': 40, 'p_max': 200, 'cost': 32, 'emissions': 0.55},
    'Wind': {'p_min': 0, 'p_max': 150, 'cost': 5, 'emissions': 0.0}
}

demand_mo = 800

# Find extreme points
min_cost_result = multi_objective_ed(generators_mo, demand_mo)
min_emissions_result = multi_objective_ed(generators_mo, demand_mo, weight_cost=0)

print(f"Extreme points:")
print(f"Min cost: ${min_cost_result['cost']:.0f}, {min_cost_result['emissions']:.1f} tons CO2")
print(f"Min emissions: ${min_emissions_result['cost']:.0f}, {min_emissions_result['emissions']:.1f} tons CO2")

# Generate Pareto frontier using epsilon-constraint
emission_range = np.linspace(min_emissions_result['emissions'], 
                           min_cost_result['emissions'], 15)
pareto_points = []

for max_emissions in emission_range:
    result = multi_objective_ed(generators_mo, demand_mo, epsilon_emissions=max_emissions)
    if result['status'] == 'optimal':
        pareto_points.append(result)

# Visualize Pareto frontier
fig = plt.figure(figsize=(15, 10))

# Pareto frontier
ax1 = plt.subplot(2, 2, 1)
costs = [p['cost'] for p in pareto_points]
emissions = [p['emissions'] for p in pareto_points]

ax1.plot(emissions, costs, 'bo-', markersize=8, linewidth=2)
ax1.set_xlabel('Total Emissions (tons CO2)')
ax1.set_ylabel('Total Cost ($)')
ax1.set_title('Pareto Frontier: Cost vs Emissions')
ax1.grid(True, alpha=0.3)

# Annotate extreme points
ax1.annotate('Min Cost', xy=(emissions[0], costs[0]), 
             xytext=(emissions[0]+10, costs[0]+200),
             arrowprops=dict(arrowstyle='->', color='red'))
ax1.annotate('Min Emissions', xy=(emissions[-1], costs[-1]), 
             xytext=(emissions[-1]-20, costs[-1]+200),
             arrowprops=dict(arrowstyle='->', color='green'))

# Generation mix along frontier
ax2 = plt.subplot(2, 2, 2)
n_points = len(pareto_points)
indices = np.linspace(0, n_points-1, 5, dtype=int)
colors = {'Coal1': '#4d4d4d', 'Coal2': '#666666', 
          'Gas1': '#ff7f0e', 'Gas2': '#ffbb78', 'Wind': '#2ca02c'}

for i, idx in enumerate(indices):
    bottom = 0
    for g in generators_mo:
        height = pareto_points[idx]['dispatch'][g]
        ax2.bar(i, height, bottom=bottom, color=colors[g], 
               label=g if i == 0 else '', edgecolor='black', linewidth=0.5)
        bottom += height

ax2.set_xlabel('Solution Index (0=Min Cost, 4=Min Emissions)')
ax2.set_ylabel('Generation (MW)')
ax2.set_title('Generation Mix Evolution')
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3, axis='y')

# Marginal trade-off
ax3 = plt.subplot(2, 2, 3)
marginal_costs = []
emission_midpoints = []

for i in range(1, len(pareto_points)):
    d_cost = pareto_points[i]['cost'] - pareto_points[i-1]['cost']
    d_emissions = pareto_points[i]['emissions'] - pareto_points[i-1]['emissions']
    if abs(d_emissions) > 0.1:
        marginal = -d_cost / d_emissions  # Negative because reducing emissions
        marginal_costs.append(marginal)
        emission_midpoints.append((pareto_points[i]['emissions'] + 
                                 pareto_points[i-1]['emissions']) / 2)

ax3.plot(emission_midpoints, marginal_costs, 'ro-', markersize=8)
ax3.set_xlabel('Emissions Level (tons CO2)')
ax3.set_ylabel('Marginal Cost ($/ton CO2)')
ax3.set_title('Marginal Cost of Emission Reduction')
ax3.grid(True, alpha=0.3)

# Efficiency analysis
ax4 = plt.subplot(2, 2, 4)
efficiency_scores = []
for p in pareto_points:
    # Simple efficiency metric: MW per unit of combined normalized objective
    normalized_obj = (p['cost'] / max(costs)) + (p['emissions'] / max(emissions))
    efficiency = demand_mo / normalized_obj
    efficiency_scores.append(efficiency)

ax4.plot(emissions, efficiency_scores, 'go-', markersize=8)
ax4.set_xlabel('Total Emissions (tons CO2)')
ax4.set_ylabel('Efficiency Score')
ax4.set_title('Solution Efficiency Along Pareto Frontier')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary table
print("\nPareto Frontier Summary (selected points):")
print("Cost ($)\tEmissions\tCoal1\tCoal2\tGas1\tGas2\tWind")
for idx in indices:
    p = pareto_points[idx]
    print(f"{p['cost']:.0f}\t{p['emissions']:.1f}\t", end='')
    for g in generators_mo:
        print(f"{p['dispatch'][g]:.0f}\t", end='')
    print()

## 3. Decomposition Techniques

Large-scale optimization problems require decomposition to remain tractable. We'll explore temporal and spatial decomposition strategies.

In [ ]:
def temporal_decomposition_uc(generators, demand_week, n_days=7, overlap_hours=4):
    """
    Solve weekly UC using temporal decomposition with rolling horizon.
    Each subproblem solves 24+overlap hours, commits 24 hours.
    """
    total_hours = n_days * 24
    all_commitment = {}
    all_dispatch = {}
    
    # Initial status
    current_status = {g: 0 for g in generators.index}  # All off initially
    current_power = {g: 0 for g in generators.index}
    
    day_results = []
    
    for day in range(n_days):
        start_hour = day * 24
        
        # Determine problem horizon
        if day < n_days - 1:
            # Include overlap hours for better decisions
            end_hour = min(start_hour + 24 + overlap_hours, total_hours)
        else:
            # Last day: solve remaining hours
            end_hour = total_hours
        
        subproblem_hours = end_hour - start_hour
        subproblem_demand = demand_week[start_hour:end_hour]
        
        # Update generator initial conditions
        generators_sub = generators.copy()
        generators_sub['initial_status'] = [current_status[g] for g in generators.index]
        generators_sub['initial_power'] = [current_power[g] for g in generators.index]
        
        # Solve subproblem
        print(f"Solving day {day+1} (hours {start_hour}-{end_hour})...", end='')
        start_time = time.time()
        
        prob = LpProblem(f"UC_Day_{day+1}", LpMinimize)
        
        # Variables
        u = {}
        p = {}
        v = {}
        
        for g in generators.index:
            for t in range(subproblem_hours):
                u[g,t] = LpVariable(f"u_{g}_{t}_d{day}", cat='Binary')
                p[g,t] = LpVariable(f"p_{g}_{t}_d{day}", 0)
                v[g,t] = LpVariable(f"v_{g}_{t}_d{day}", cat='Binary')
        
        # Simplified objective
        prob += lpSum(
            generators_sub.loc[g, 'cost'] * p[g,t] + 
            generators_sub.loc[g, 'startup_cost'] * v[g,t]
            for g in generators.index for t in range(subproblem_hours)
        )
        
        # Basic constraints
        for t in range(subproblem_hours):
            # Demand
            prob += lpSum(p[g,t] for g in generators.index) == subproblem_demand[t]
            
            # Generation limits
            for g in generators.index:
                prob += p[g,t] >= generators_sub.loc[g, 'p_min'] * u[g,t]
                prob += p[g,t] <= generators_sub.loc[g, 'p_max'] * u[g,t]
                
                # Startup logic
                if t == 0:
                    prob += v[g,t] >= u[g,t] - generators_sub.loc[g, 'initial_status']
                else:
                    prob += v[g,t] >= u[g,t] - u[g,t-1]
        
        # Solve
        prob.solve(PULP_CBC_CMD(msg=0, timeLimit=30))
        solve_time = time.time() - start_time
        print(f" solved in {solve_time:.1f}s")
        
        # Store committed results (first 24 hours only)
        commit_hours = min(24, subproblem_hours)
        for t in range(commit_hours):
            global_t = start_hour + t
            for g in generators.index:
                all_commitment[g, global_t] = value(u[g,t])
                all_dispatch[g, global_t] = value(p[g,t])
        
        # Update status for next iteration
        for g in generators.index:
            current_status[g] = value(u[g,commit_hours-1])
            current_power[g] = value(p[g,commit_hours-1])
        
        day_results.append({
            'day': day + 1,
            'cost': value(prob.objective),
            'solve_time': solve_time
        })
    
    return all_commitment, all_dispatch, day_results

# Create weekly demand profile
daily_pattern = demand  # From earlier
weekly_demand = []
for day in range(7):
    # Add daily variation
    if day < 5:  # Weekday
        scale = 1.0 + 0.05 * np.random.randn()
    else:  # Weekend
        scale = 0.9 + 0.05 * np.random.randn()
    weekly_demand.extend(daily_pattern * scale)

# Solve using decomposition
print("Temporal Decomposition for Weekly Unit Commitment:")
commitment, dispatch, day_results = temporal_decomposition_uc(generators, weekly_demand)

# Visualize results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))

# Weekly generation profile
hours = range(168)
colors = plt.cm.Set3(np.linspace(0, 1, len(generators)))
bottom = np.zeros(168)

for i, g in enumerate(generators.index):
    gen_profile = [dispatch[g,t] if (g,t) in dispatch else 0 for t in hours]
    ax1.fill_between(hours, bottom, bottom + gen_profile, 
                     label=g, color=colors[i], alpha=0.7)
    bottom += gen_profile

ax1.plot(hours, weekly_demand, 'k--', linewidth=2, label='Demand')
ax1.set_xlabel('Hour of Week')
ax1.set_ylabel('Power (MW)')
ax1.set_title('Weekly Generation Schedule (Temporal Decomposition)')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Add day boundaries
for day in range(1, 7):
    ax1.axvline(x=day*24, color='gray', linestyle=':', alpha=0.5)
    ax1.text(day*24-12, ax1.get_ylim()[1]*0.95, f'Day {day}', 
            ha='center', fontsize=10)

# Computational performance
days = [r['day'] for r in day_results]
solve_times = [r['solve_time'] for r in day_results]

ax2.bar(days, solve_times, color='skyblue', edgecolor='navy', alpha=0.7)
ax2.set_xlabel('Day')
ax2.set_ylabel('Solve Time (seconds)')
ax2.set_title('Computational Performance by Day')
ax2.grid(True, alpha=0.3, axis='y')

# Add cumulative time
cumulative_time = np.cumsum(solve_times)
ax2_twin = ax2.twinx()
ax2_twin.plot(days, cumulative_time, 'ro-', linewidth=2, markersize=8)
ax2_twin.set_ylabel('Cumulative Time (seconds)', color='r')
ax2_twin.tick_params(axis='y', labelcolor='r')

plt.tight_layout()
plt.show()

print(f"\nTotal solve time: {sum(solve_times):.1f} seconds")
print(f"Average time per day: {np.mean(solve_times):.1f} seconds")

### Exercise 3: Benders Decomposition

Implement a simplified version of Benders decomposition for a two-stage stochastic unit commitment problem.

In [ ]:
# Exercise 3: Implement Benders decomposition
# Requirements:
# - Master problem: unit commitment decisions
# - Subproblems: economic dispatch for each scenario
# - Add Benders cuts iteratively
# - Compare with extensive form solution

# Your implementation here

In [ ]:
# Solution
def benders_decomposition(generators, demand, scenarios, max_iterations=10, gap=0.01):
    """Simplified Benders decomposition for stochastic UC."""
    n_gen = len(generators)
    n_scenarios = len(scenarios)
    
    # Initialize
    lower_bound = -np.inf
    upper_bound = np.inf
    iteration = 0
    cuts = []
    
    results_history = []
    
    while iteration < max_iterations and (upper_bound - lower_bound) / upper_bound > gap:
        iteration += 1
        print(f"\nIteration {iteration}:")
        
        # Master Problem: Unit commitment
        master = LpProblem(f"Master_{iteration}", LpMinimize)
        
        # First-stage variables
        u = {g: LpVariable(f"u_{g}", cat='Binary') for g in generators.index}
        theta = {s: LpVariable(f"theta_{s}", lowBound=0) for s in scenarios}  # Recourse cost
        
        # Objective: startup costs + expected recourse
        master += lpSum(generators.loc[g, 'startup_cost'] * u[g] for g in generators.index) + \
                 lpSum(theta[s] / n_scenarios for s in scenarios)
        
        # Add Benders cuts from previous iterations
        for cut in cuts:
            master += cut
        
        # Basic constraints (simplified)
        master += lpSum(generators.loc[g, 'p_max'] * u[g] for g in generators.index) >= max(demand) * 1.1
        
        # Solve master
        master.solve()
        
        if master.status != 1:
            print("Master problem infeasible!")
            break
        
        # Get commitment decisions
        u_sol = {g: value(u[g]) for g in generators.index}
        master_obj = value(master.objective)
        print(f"Master objective: {master_obj:.0f}")
        
        # Solve subproblems for each scenario
        sub_objectives = []
        sub_duals = []
        
        for s_idx, (s_name, s_demand) in enumerate(scenarios.items()):
            # Subproblem: Economic dispatch given commitment
            sub = LpProblem(f"Sub_{iteration}_{s_name}", LpMinimize)
            
            # Second-stage variables
            p = {g: LpVariable(f"p_{g}_{s_name}", 0) for g in generators.index}
            slack = LpVariable(f"slack_{s_name}", 0)  # Penalty for unmet demand
            
            # Objective: generation cost + penalty
            sub += lpSum(generators.loc[g, 'cost'] * p[g] for g in generators.index) + \
                   10000 * slack  # High penalty
            
            # Constraints
            demand_constraint = sub.addConstraint(
                lpSum(p[g] for g in generators.index) + slack >= s_demand[12],  # Hour 12
                f"Demand_{s_name}"
            )
            
            # Generation limits based on commitment
            for g in generators.index:
                if u_sol[g] > 0.5:
                    sub += p[g] <= generators.loc[g, 'p_max']
                    sub += p[g] >= generators.loc[g, 'p_min']
                else:
                    sub += p[g] == 0
            
            # Solve subproblem
            sub.solve()
            
            if sub.status == 1:
                sub_obj = value(sub.objective)
                sub_objectives.append(sub_obj)
                
                # Get dual values (simplified - using slack penalty as proxy)
                if value(slack) > 0.01:
                    # Infeasible - need more capacity
                    dual_approx = 10000
                else:
                    # Feasible - marginal cost
                    dual_approx = max(generators.loc[g, 'cost'] 
                                    for g in generators.index if value(p[g]) > 0.01)
                sub_duals.append(dual_approx)
            else:
                sub_objectives.append(np.inf)
                sub_duals.append(10000)
        
        # Update bounds
        recourse_cost = sum(sub_objectives) / n_scenarios
        current_ub = sum(generators.loc[g, 'startup_cost'] * u_sol[g] 
                        for g in generators.index) + recourse_cost
        upper_bound = min(upper_bound, current_ub)
        lower_bound = master_obj
        
        print(f"Subproblem avg cost: {recourse_cost:.0f}")
        print(f"Lower bound: {lower_bound:.0f}, Upper bound: {upper_bound:.0f}")
        print(f"Gap: {(upper_bound - lower_bound) / upper_bound * 100:.1f}%")
        
        # Generate Benders cut
        # Simplified: theta[s] >= Q(u_sol, s) + sensitivity * (u - u_sol)
        for s_idx, s_name in enumerate(scenarios.keys()):
            # Approximate subgradient
            cut_expr = sub_objectives[s_idx]
            for g in generators.index:
                sensitivity = sub_duals[s_idx] * generators.loc[g, 'p_min'] * 0.5
                if u_sol[g] > 0.5:
                    cut_expr += sensitivity * (1 - u[g])
                else:
                    cut_expr += sensitivity * u[g]
            
            new_cut = theta[s_name] >= cut_expr
            cuts.append(new_cut)
            master += new_cut
        
        results_history.append({
            'iteration': iteration,
            'lower_bound': lower_bound,
            'upper_bound': upper_bound,
            'gap': (upper_bound - lower_bound) / upper_bound,
            'commitment': u_sol.copy()
        })
    
    return results_history, u_sol

# Create scenarios for Benders
demand_scenarios = {}
base_demand_profile = demand
for i in range(5):
    scale = 0.8 + 0.4 * np.random.rand()
    demand_scenarios[f'S{i+1}'] = base_demand_profile * scale

# Run Benders decomposition
print("Benders Decomposition for Stochastic Unit Commitment:")
benders_results, final_commitment = benders_decomposition(
    generators, base_demand_profile, demand_scenarios
)

# Visualize convergence
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Bounds convergence
iterations = [r['iteration'] for r in benders_results]
lower_bounds = [r['lower_bound'] for r in benders_results]
upper_bounds = [r['upper_bound'] for r in benders_results]

ax1.plot(iterations, lower_bounds, 'b-', marker='o', label='Lower Bound', markersize=8)
ax1.plot(iterations, upper_bounds, 'r-', marker='s', label='Upper Bound', markersize=8)
ax1.fill_between(iterations, lower_bounds, upper_bounds, alpha=0.3, color='gray')
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Objective Value ($)')
ax1.set_title('Benders Decomposition Convergence')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gap evolution
gaps = [r['gap'] * 100 for r in benders_results]
ax2.plot(iterations, gaps, 'g-', marker='D', markersize=8)
ax2.axhline(y=1, color='r', linestyle='--', label='1% Target')
ax2.set_xlabel('Iteration')
ax2.set_ylabel('Optimality Gap (%)')
ax2.set_title('Gap Reduction')
ax2.set_yscale('log')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Final solution
print("\nFinal Unit Commitment:")
for g in generators.index:
    status = "ON" if final_commitment[g] > 0.5 else "OFF"
    print(f"{g}: {status}")

# Compare with extensive form (if computationally feasible)
print("\nComparison with Extensive Form:")
print("Benders: Iterative solution with cuts")
print("Extensive: Would require solving all scenarios simultaneously")
print(f"Problem size reduction: {len(generators) * len(demand_scenarios) * 24} variables → "
      f"{len(generators) + len(demand_scenarios)} master variables per iteration")

## 4. Case Study: Storage Integration

Energy storage fundamentally changes power system optimization by decoupling generation and consumption. We'll develop a comprehensive storage model.

### Exercise 4: Storage Co-optimization

Integrate battery storage into the unit commitment problem, considering energy arbitrage, reserves provision, and degradation costs.

In [ ]:
# Exercise 4: Add storage to unit commitment
# Requirements:
# - 100 MW / 400 MWh battery system
# - Round-trip efficiency: 85%
# - Degradation cost: $5/MWh cycled
# - Can provide both energy and reserves
# - Compare system cost with and without storage

# Your implementation here

In [ ]:
# Solution
def uc_with_storage(generators, demand, storage_params, wind_profile=None):
    """UC with battery storage integration."""
    T = len(demand)
    periods = range(T)
    gen_names = generators.index.tolist()
    
    # Storage parameters
    p_charge_max = storage_params['power_mw']
    p_discharge_max = storage_params['power_mw']
    e_capacity = storage_params['energy_mwh']
    efficiency = storage_params['efficiency']
    degradation_cost = storage_params['degradation_cost']
    initial_soc = storage_params['initial_soc'] * e_capacity
    
    # Create problem
    prob = LpProblem("UC_Storage", LpMinimize)
    
    # Generator variables
    u = {}
    p = {}
    v = {}
    
    for g in gen_names:
        for t in periods:
            u[g,t] = LpVariable(f"u_{g}_{t}", cat='Binary')
            p[g,t] = LpVariable(f"p_{g}_{t}", 0)
            v[g,t] = LpVariable(f"v_{g}_{t}", cat='Binary')
    
    # Storage variables
    p_charge = {t: LpVariable(f"p_charge_{t}", 0, p_charge_max) for t in periods}
    p_discharge = {t: LpVariable(f"p_discharge_{t}", 0, p_discharge_max) for t in periods}
    soc = {t: LpVariable(f"soc_{t}", 0, e_capacity) for t in periods}
    
    # Reserve from storage
    r_storage = {t: LpVariable(f"r_storage_{t}", 0, p_discharge_max) for t in periods}
    
    # Objective
    gen_cost = lpSum(generators.loc[g, 'cost'] * p[g,t] + 
                     generators.loc[g, 'startup_cost'] * v[g,t]
                     for g in gen_names for t in periods)
    
    storage_cost = lpSum(degradation_cost * (p_charge[t] + p_discharge[t]) 
                        for t in periods)
    
    prob += gen_cost + storage_cost
    
    # Constraints
    for t in periods:
        # Power balance
        total_gen = lpSum(p[g,t] for g in gen_names)
        if wind_profile is not None:
            total_gen += wind_profile[t]
        
        prob += total_gen + p_discharge[t] == demand[t] + p_charge[t], f"Balance_{t}"
        
        # Reserve (including storage)
        gen_reserve = lpSum(generators.loc[g, 'p_max'] * u[g,t] - p[g,t] 
                           for g in gen_names)
        prob += gen_reserve + r_storage[t] >= 0.1 * demand[t], f"Reserve_{t}"
        
        # Storage reserve limit
        prob += r_storage[t] + p_discharge[t] <= p_discharge_max, f"Storage_reserve_{t}"
        prob += r_storage[t] <= soc[t], f"Storage_energy_reserve_{t}"  # Must have energy
        
        # State of charge dynamics
        if t == 0:
            prob += soc[t] == initial_soc + \
                    efficiency * p_charge[t] - p_discharge[t], f"SOC_{t}"
        else:
            prob += soc[t] == soc[t-1] + \
                    efficiency * p_charge[t] - p_discharge[t], f"SOC_{t}"
        
        # Cannot charge and discharge simultaneously (linear approximation)
        prob += p_charge[t] + p_discharge[t] <= p_charge_max, f"Storage_excl_{t}"
    
    # End-of-day SOC constraint (cyclical operation)
    prob += soc[T-1] >= 0.5 * initial_soc, "Final_SOC_min"
    prob += soc[T-1] <= 1.5 * initial_soc, "Final_SOC_max"
    
    # Generator constraints (simplified)
    for g in gen_names:
        for t in periods:
            prob += p[g,t] >= generators.loc[g, 'p_min'] * u[g,t]
            prob += p[g,t] <= generators.loc[g, 'p_max'] * u[g,t]
            
            if t == 0:
                prob += v[g,t] >= u[g,t] - generators.loc[g, 'initial_status']
            else:
                prob += v[g,t] >= u[g,t] - u[g,t-1]
    
    # Solve
    prob.solve(PULP_CBC_CMD(timeLimit=60, msg=0))
    
    return prob, p, p_charge, p_discharge, soc, r_storage

# Storage parameters
storage = {
    'power_mw': 100,
    'energy_mwh': 400,
    'efficiency': 0.85,
    'degradation_cost': 5,
    'initial_soc': 0.5
}

# Add wind profile
wind_capacity = 150
wind_profile = wind_capacity * (0.3 + 0.4 * np.sin(2 * np.pi * np.arange(24) / 24) + 
                               0.2 * np.random.randn(24))
wind_profile = np.maximum(wind_profile, 0)

# Solve with and without storage
print("Solving UC without storage...")
prob_no_storage, p_no_storage, _, _, _, _ = uc_with_storage(
    generators, demand, 
    {'power_mw': 0, 'energy_mwh': 0, 'efficiency': 1, 'degradation_cost': 0, 'initial_soc': 0},
    wind_profile
)

print("Solving UC with storage...")
prob_storage, p_storage, p_charge, p_discharge, soc, r_storage = uc_with_storage(
    generators, demand, storage, wind_profile
)

# Compare results
cost_no_storage = value(prob_no_storage.objective) if prob_no_storage.status == 1 else np.inf
cost_storage = value(prob_storage.objective) if prob_storage.status == 1 else np.inf
savings = cost_no_storage - cost_storage

print(f"\nCost without storage: ${cost_no_storage:,.0f}")
print(f"Cost with storage: ${cost_storage:,.0f}")
print(f"Savings: ${savings:,.0f} ({savings/cost_no_storage*100:.1f}%)")

# Visualize storage operation
fig = plt.figure(figsize=(16, 12))

# Generation comparison
ax1 = plt.subplot(3, 2, 1)
hours = range(24)
colors = plt.cm.Set3(np.linspace(0, 1, len(generators) + 2))

# Without storage
bottom = wind_profile.copy()
ax1.fill_between(hours, 0, bottom, color='lightgreen', alpha=0.7, label='Wind')
for i, g in enumerate(generators.index):
    gen_output = [value(p_no_storage[g,t]) for t in hours]
    ax1.fill_between(hours, bottom, bottom + gen_output, 
                    color=colors[i], alpha=0.7, label=g)
    bottom += gen_output

ax1.plot(hours, demand, 'k--', linewidth=2, label='Demand')
ax1.set_ylabel('Power (MW)')
ax1.set_title('Generation Without Storage')
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3)

# With storage
ax2 = plt.subplot(3, 2, 2)
bottom = wind_profile.copy()
ax2.fill_between(hours, 0, bottom, color='lightgreen', alpha=0.7, label='Wind')
for i, g in enumerate(generators.index):
    gen_output = [value(p_storage[g,t]) for t in hours]
    ax2.fill_between(hours, bottom, bottom + gen_output, 
                    color=colors[i], alpha=0.7, label=g)
    bottom += gen_output

# Add storage discharge
discharge = [value(p_discharge[t]) for t in hours]
ax2.fill_between(hours, bottom, bottom + discharge, 
                color='red', alpha=0.5, label='Storage Discharge')

ax2.plot(hours, demand, 'k--', linewidth=2, label='Demand')
ax2.set_ylabel('Power (MW)')
ax2.set_title('Generation With Storage')
ax2.legend(fontsize=8)
ax2.grid(True, alpha=0.3)

# Storage operation
ax3 = plt.subplot(3, 2, 3)
charge = [value(p_charge[t]) for t in hours]
discharge_neg = [-d for d in discharge]

ax3.bar(hours, charge, color='blue', alpha=0.7, label='Charging')
ax3.bar(hours, discharge_neg, color='red', alpha=0.7, label='Discharging')
ax3.axhline(y=0, color='black', linewidth=1)
ax3.set_xlabel('Hour')
ax3.set_ylabel('Storage Power (MW)')
ax3.set_title('Storage Charge/Discharge Profile')
ax3.legend()
ax3.grid(True, alpha=0.3)

# State of charge
ax4 = plt.subplot(3, 2, 4)
soc_values = [value(soc[t]) for t in hours]
ax4.plot(hours, soc_values, 'g-', linewidth=3)
ax4.axhline(y=storage['energy_mwh'], color='r', linestyle='--', label='Capacity')
ax4.fill_between(hours, 0, soc_values, alpha=0.3, color='green')
ax4.set_xlabel('Hour')
ax4.set_ylabel('State of Charge (MWh)')
ax4.set_title('Battery Energy Level')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Price signals (proxy)
ax5 = plt.subplot(3, 2, 5)
marginal_costs = []
for t in hours:
    # Estimate marginal cost based on most expensive online unit
    online_costs = [generators.loc[g, 'cost'] 
                   for g in generators.index 
                   if value(p_storage[g,t]) > 0.1]
    marginal_costs.append(max(online_costs) if online_costs else 0)

ax5.plot(hours, marginal_costs, 'b-', linewidth=2, label='Marginal Cost')
ax5_twin = ax5.twinx()
net_storage = [d - c for d, c in zip(discharge, charge)]
ax5_twin.bar(hours, net_storage, alpha=0.5, color='gray', label='Net Storage')
ax5.set_xlabel('Hour')
ax5.set_ylabel('Marginal Cost ($/MWh)', color='b')
ax5_twin.set_ylabel('Net Storage (MW)', color='gray')
ax5.set_title('Storage Arbitrage Opportunity')
ax5.grid(True, alpha=0.3)

# Value streams
ax6 = plt.subplot(3, 2, 6)
# Calculate value components
energy_value = sum((marginal_costs[t] - storage['degradation_cost']) * discharge[t] 
                  for t in hours)
reserve_value = sum(30 * value(r_storage[t]) for t in hours)  # Assume $30/MW for reserves
cycling_cost = sum(storage['degradation_cost'] * (charge[t] + discharge[t]) 
                  for t in hours)

values = [energy_value, reserve_value, -cycling_cost]
labels = ['Energy\nArbitrage', 'Reserve\nProvision', 'Cycling\nCost']
colors_bar = ['green', 'blue', 'red']

bars = ax6.bar(labels, values, color=colors_bar, alpha=0.7)
ax6.axhline(y=0, color='black', linewidth=1)
ax6.set_ylabel('Value ($)')
ax6.set_title('Storage Value Breakdown (24 hours)')
ax6.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, val in zip(bars, values):
    height = bar.get_height()
    ax6.text(bar.get_x() + bar.get_width()/2., height + np.sign(height) * 50,
             f'${val:.0f}', ha='center', va='bottom' if val > 0 else 'top')

plt.tight_layout()
plt.show()

# Storage utilization summary
total_throughput = sum(discharge[t] for t in hours)
cycles = total_throughput / storage['energy_mwh']
capacity_factor = total_throughput / (storage['power_mw'] * 24) * 100

print(f"\nStorage Operation Summary:")
print(f"Total energy discharged: {total_throughput:.0f} MWh")
print(f"Equivalent full cycles: {cycles:.2f}")
print(f"Capacity factor: {capacity_factor:.1f}%")
print(f"Average round-trip efficiency realized: {sum(discharge)/sum(charge)*100:.1f}%")

## 5. Case Study: Transmission Planning

Transmission expansion planning involves long-term investment decisions considering uncertain future generation and demand patterns.

### Exercise 5: Transmission Expansion

Solve a transmission expansion problem that minimizes investment costs plus expected operational costs over multiple scenarios.

In [ ]:
# Exercise 5: Transmission expansion planning
# Requirements:
# - 3-bus system with potential new lines
# - Investment cost vs operational savings
# - Consider multiple demand growth scenarios
# - Include N-1 reliability constraints

# Your implementation here

In [ ]:
# Solution
def transmission_expansion_planning(buses, generators, existing_lines, candidate_lines, 
                                  scenarios, n1_contingencies=True):
    """Multi-stage transmission expansion with operational considerations."""
    prob = LpProblem("Transmission_Expansion", LpMinimize)
    
    # Investment variables (first stage)
    x = {l: LpVariable(f"build_{l}", cat='Binary') for l in candidate_lines.index}
    
    # Operational variables (second stage) for each scenario
    p_gen = {}
    theta = {}
    flow = {}
    
    for s in scenarios:
        for g in generators.index:
            p_gen[s,g] = LpVariable(f"p_{s}_{g}", 0, generators.loc[g, 'p_max'])
        
        for b in buses.index:
            theta[s,b] = LpVariable(f"theta_{s}_{b}", -np.pi, np.pi)
        
        # Flows on all lines (existing + candidates)
        all_lines = list(existing_lines.index) + list(candidate_lines.index)
        for l in all_lines:
            flow[s,l] = LpVariable(f"flow_{s}_{l}")
    
    # Objective: investment + expected operational cost
    investment_cost = lpSum(candidate_lines.loc[l, 'cost'] * x[l] for l in candidate_lines.index)
    
    operational_cost = lpSum(
        scenarios[s]['probability'] * scenarios[s]['weight_years'] * 
        lpSum(generators.loc[g, 'cost'] * p_gen[s,g] for g in generators.index)
        for s in scenarios
    )
    
    prob += investment_cost + operational_cost
    
    # Constraints for each scenario
    for s in scenarios:
        # Reference bus
        prob += theta[s, 'Bus1'] == 0
        
        # Power balance at each bus
        for b in buses.index:
            # Generation at bus
            gen_at_bus = lpSum(p_gen[s,g] for g in generators.index 
                              if generators.loc[g, 'bus'] == b)
            
            # Net injection
            injection = gen_at_bus - scenarios[s]['demand'][b]
            
            # Power flow balance
            flow_out = 0
            
            # Existing lines
            for l in existing_lines.index:
                if existing_lines.loc[l, 'from_bus'] == b:
                    flow_out += flow[s,l]
                elif existing_lines.loc[l, 'to_bus'] == b:
                    flow_out -= flow[s,l]
            
            # Candidate lines
            for l in candidate_lines.index:
                if candidate_lines.loc[l, 'from_bus'] == b:
                    flow_out += flow[s,l]
                elif candidate_lines.loc[l, 'to_bus'] == b:
                    flow_out -= flow[s,l]
            
            prob += injection == flow_out, f"Balance_{s}_{b}"
        
        # DC power flow and limits for existing lines
        for l in existing_lines.index:
            line = existing_lines.loc[l]
            from_bus = line['from_bus']
            to_bus = line['to_bus']
            
            # DC power flow
            prob += (flow[s,l] == 
                    (theta[s,from_bus] - theta[s,to_bus]) / line['reactance'],
                    f"DC_flow_{s}_{l}")
            
            # Flow limits
            prob += flow[s,l] <= line['limit'], f"Limit_pos_{s}_{l}"
            prob += flow[s,l] >= -line['limit'], f"Limit_neg_{s}_{l}"
        
        # DC power flow and limits for candidate lines
        for l in candidate_lines.index:
            line = candidate_lines.loc[l]
            from_bus = line['from_bus']
            to_bus = line['to_bus']
            
            # DC power flow (with big-M)
            M = 10  # Big-M constant
            prob += (flow[s,l] <= 
                    (theta[s,from_bus] - theta[s,to_bus]) / line['reactance'] + M * (1 - x[l]),
                    f"DC_flow_upper_{s}_{l}")
            prob += (flow[s,l] >= 
                    (theta[s,from_bus] - theta[s,to_bus]) / line['reactance'] - M * (1 - x[l]),
                    f"DC_flow_lower_{s}_{l}")
            
            # Flow limits (only if built)
            prob += flow[s,l] <= line['limit'] * x[l], f"Limit_pos_{s}_{l}"
            prob += flow[s,l] >= -line['limit'] * x[l], f"Limit_neg_{s}_{l}"
        
        # N-1 contingency constraints (simplified)
        if n1_contingencies:
            # Only check critical contingencies
            for cont_line in existing_lines.index[:1]:  # First line as example
                # Approximate post-contingency flows
                for l in existing_lines.index:
                    if l != cont_line:
                        # Simple approximation: 20% flow increase
                        prob += flow[s,l] <= 0.8 * existing_lines.loc[l, 'limit'], \
                                f"N1_cont_{s}_{l}_out_{cont_line}"
    
    # Solve
    prob.solve(PULP_CBC_CMD(timeLimit=60, msg=0))
    
    return prob, x, p_gen, flow

# Define system
buses_tep = pd.DataFrame(index=['Bus1', 'Bus2', 'Bus3'])

generators_tep = pd.DataFrame({
    'bus': ['Bus1', 'Bus2', 'Bus3'],
    'p_max': [300, 200, 250],
    'cost': [25, 30, 28]
}, index=['G1', 'G2', 'G3'])

existing_lines_tep = pd.DataFrame({
    'from_bus': ['Bus1', 'Bus2'],
    'to_bus': ['Bus2', 'Bus3'],
    'limit': [150, 100],
    'reactance': [0.1, 0.15]
}, index=['L1', 'L2'])

candidate_lines_tep = pd.DataFrame({
    'from_bus': ['Bus1', 'Bus1', 'Bus2'],
    'to_bus': ['Bus3', 'Bus2', 'Bus3'],
    'limit': [200, 150, 150],
    'reactance': [0.12, 0.1, 0.15],
    'cost': [50e6, 30e6, 40e6]  # Million $
}, index=['C1', 'C2', 'C3'])

# Scenarios
scenarios_tep = {
    'Low': {
        'probability': 0.3,
        'weight_years': 8760 * 10,  # 10 years of hours
        'demand': {'Bus1': 0, 'Bus2': 200, 'Bus3': 150}
    },
    'Base': {
        'probability': 0.5,
        'weight_years': 8760 * 10,
        'demand': {'Bus1': 0, 'Bus2': 250, 'Bus3': 200}
    },
    'High': {
        'probability': 0.2,
        'weight_years': 8760 * 10,
        'demand': {'Bus1': 0, 'Bus2': 300, 'Bus3': 250}
    }
}

# Solve expansion problem
print("Solving Transmission Expansion Planning...")
prob_tep, x_tep, p_gen_tep, flow_tep = transmission_expansion_planning(
    buses_tep, generators_tep, existing_lines_tep, candidate_lines_tep, scenarios_tep
)

print(f"\nOptimal Solution:")
print(f"Status: {LpStatus[prob_tep.status]}")
print(f"Total cost: ${value(prob_tep.objective)/1e6:.1f} million")

# Investment decisions
investment_total = 0
built_lines = []
print("\nTransmission investments:")
for l in candidate_lines_tep.index:
    if value(x_tep[l]) > 0.5:
        print(f"Build {l}: {candidate_lines_tep.loc[l, 'from_bus']} → "
              f"{candidate_lines_tep.loc[l, 'to_bus']} "
              f"(${candidate_lines_tep.loc[l, 'cost']/1e6:.0f}M)")
        investment_total += candidate_lines_tep.loc[l, 'cost']
        built_lines.append(l)

operational_total = (value(prob_tep.objective) - investment_total) / 1e6
print(f"\nCost breakdown:")
print(f"Investment cost: ${investment_total/1e6:.1f}M")
print(f"Expected operational cost (NPV): ${operational_total:.1f}M")

# Visualize network and flows
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Network positions
pos = {'Bus1': (0, 1), 'Bus2': (1, 0), 'Bus3': (1, 2)}

for idx, (s_name, s_data) in enumerate(scenarios_tep.items()):
    ax = axes[idx]
    
    # Draw buses
    for bus, (x, y) in pos.items():
        circle = plt.Circle((x, y), 0.15, color='lightblue', ec='black', linewidth=2)
        ax.add_patch(circle)
        ax.text(x, y, bus, ha='center', va='center', fontweight='bold')
        
        # Demand
        if s_data['demand'][bus] > 0:
            ax.text(x, y-0.25, f"D: {s_data['demand'][bus]}MW", 
                   ha='center', fontsize=9)
    
    # Draw existing lines
    for _, line in existing_lines_tep.iterrows():
        x1, y1 = pos[line['from_bus']]
        x2, y2 = pos[line['to_bus']]
        ax.plot([x1, x2], [y1, y2], 'k-', linewidth=2)
        
        # Flow
        flow_val = value(flow_tep[s_name, line.name])
        mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
        ax.text(mid_x - 0.15, mid_y, f"{flow_val:.0f}MW", 
               fontsize=8, bbox=dict(boxstyle="round,pad=0.3", 
                                   facecolor="white", alpha=0.8))
    
    # Draw built candidate lines
    for l in built_lines:
        line = candidate_lines_tep.loc[l]
        x1, y1 = pos[line['from_bus']]
        x2, y2 = pos[line['to_bus']]
        ax.plot([x1, x2], [y1, y2], 'g--', linewidth=3, alpha=0.7)
        
        # Flow
        flow_val = value(flow_tep[s_name, l])
        mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
        ax.text(mid_x + 0.15, mid_y, f"{flow_val:.0f}MW", 
               fontsize=8, color='green', fontweight='bold',
               bbox=dict(boxstyle="round,pad=0.3", 
                        facecolor="lightgreen", alpha=0.8))
    
    # Generation
    for g in generators_tep.index:
        bus = generators_tep.loc[g, 'bus']
        x, y = pos[bus]
        gen_val = value(p_gen_tep[s_name, g])
        ax.text(x, y + 0.25, f"G: {gen_val:.0f}MW", 
               ha='center', fontsize=9, color='red', fontweight='bold')
    
    ax.set_xlim(-0.5, 1.5)
    ax.set_ylim(-0.5, 2.5)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(f'{s_name} Demand Scenario\n(Prob: {s_data["probability"]})')

# Add legend
axes[1].plot([], [], 'k-', linewidth=2, label='Existing Line')
axes[1].plot([], [], 'g--', linewidth=3, label='New Line')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05))

plt.suptitle('Transmission Expansion Solution Across Scenarios', fontsize=16)
plt.tight_layout()
plt.show()

# Economic analysis
print("\nEconomic Analysis:")
for l in built_lines:
    congestion_relief = 0
    for s in scenarios_tep:
        # Estimate value from flow
        flow_value = abs(value(flow_tep[s, l])) * 30 * scenarios_tep[s]['weight_years']
        congestion_relief += scenarios_tep[s]['probability'] * flow_value
    
    payback = candidate_lines_tep.loc[l, 'cost'] / (congestion_relief / 10)  # Annual
    print(f"{l}: Payback period ≈ {payback:.1f} years")

## 6. Future Grid Optimization Framework

Design an optimization framework for a future grid with high renewable penetration, widespread storage, and flexible demand.

In [ ]:
def future_grid_framework():
    """
    Conceptual framework for future grid optimization.
    Demonstrates structure rather than full implementation.
    """
    framework = {
        'objectives': [
            'Minimize total system cost',
            'Minimize carbon emissions',
            'Maximize renewable utilization',
            'Ensure reliability standards'
        ],
        'decision_variables': {
            'operational': [
                'Conventional generation dispatch',
                'Renewable curtailment',
                'Storage charge/discharge',
                'Flexible demand scheduling',
                'Virtual power plant aggregation'
            ],
            'investment': [
                'Renewable capacity expansion',
                'Storage deployment',
                'Transmission reinforcement',
                'Grid flexibility resources'
            ]
        },
        'constraints': {
            'technical': [
                'Power balance with uncertainty',
                'Voltage and frequency limits',
                'Ramping and flexibility requirements',
                'Storage state-of-charge limits'
            ],
            'reliability': [
                'Probabilistic resource adequacy',
                'N-1-1 contingency coverage',
                'Resilience to extreme events'
            ],
            'market': [
                'Locational marginal pricing',
                'Capacity market requirements',
                'Ancillary service provision'
            ]
        },
        'uncertainty_handling': {
            'sources': [
                'Renewable generation (wind, solar)',
                'Demand patterns with electrification',
                'Electric vehicle charging',
                'Distributed resource availability'
            ],
            'methods': [
                'Adaptive robust optimization',
                'Distributionally robust optimization',
                'Multi-stage stochastic programming',
                'Machine learning forecasts'
            ]
        },
        'computational_approach': {
            'decomposition': [
                'Temporal: multi-timescale coordination',
                'Spatial: distributed optimization',
                'Scenario: progressive hedging'
            ],
            'acceleration': [
                'Warm starting from forecasts',
                'Neural network approximations',
                'Parallel computing',
                'Quantum optimization potential'
            ]
        }
    }
    
    return framework

# Display framework
framework = future_grid_framework()

print("Future Grid Optimization Framework")
print("=" * 50)

for category, content in framework.items():
    print(f"\n{category.upper().replace('_', ' ')}:")
    if isinstance(content, list):
        for item in content:
            print(f"  • {item}")
    else:
        for subcategory, items in content.items():
            print(f"  {subcategory.title()}:")
            for item in items:
                print(f"    - {item}")

# Conceptual implementation sketch
print("\n" + "=" * 50)
print("Implementation Roadmap:")
print("=" * 50)

roadmap = [
    "Phase 1: Enhanced stochastic unit commitment with storage",
    "Phase 2: Distribution-transmission co-optimization",
    "Phase 3: Real-time flexibility market integration",
    "Phase 4: AI-assisted optimization with human oversight",
    "Phase 5: Fully autonomous grid operation with resilience"
]

for i, phase in enumerate(roadmap, 1):
    print(f"{i}. {phase}")

## Summary

This lesson explored advanced optimization techniques essential for modern and future power systems:

1. **Uncertainty Handling**: Stochastic, robust, and chance-constrained approaches for renewable integration
2. **Multi-Objective Optimization**: Balancing cost, emissions, and reliability through Pareto analysis
3. **Decomposition Methods**: Temporal and spatial decomposition for computational tractability
4. **Storage Integration**: Co-optimizing energy and ancillary services from storage resources
5. **Transmission Planning**: Long-term investment decisions under uncertainty
6. **Future Frameworks**: Designing optimization systems for high-renewable grids

These advanced techniques build on the fundamental optimization concepts to address real-world complexity. As power systems evolve with more renewable resources, storage, and active demand participation, sophisticated optimization becomes even more critical for maintaining reliability and efficiency.

The key insight is that no single optimization technique solves all problems. Successful power system optimization requires:
- Understanding the problem structure
- Choosing appropriate modeling approximations
- Balancing solution quality with computational requirements
- Validating results against engineering intuition

These skills prepare you to tackle novel optimization challenges as the grid continues its transformation.